In [1]:
# ! pip uninstall -y nemo_guardrails

# Trace Recording Example

In [2]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [3]:
from trulens_eval.trace import Tracer, TracerProvider, tracer_provider
tp = tracer_provider

with tp.trace() as tracer:
    with tracer.method() as span1:
        print(type(span1), span1)

    with tp.trace() as inner_tracer:

        with tracer.method() as span2:
            print(type(span2), span2)
            raise Exception("test outer tracer error")

        with inner_tracer.method() as span3:
            print(type(span3), span3)
            raise Exception("test inner tracer error")

tracer SpanRecord
tracer SpanMethodCall
<class 'trulens_eval.trace.SpanMethodCall'> context=eb/03 parent=eb/0e error=None call=None
tracer SpanRecord
tracer SpanMethodCall
<class 'trulens_eval.trace.SpanMethodCall'> context=eb/eb parent=eb/0e error=None call=None
tracer SpanMethodCall
<class 'trulens_eval.trace.SpanMethodCall'> context=a5/e4 parent=a5/66 error=None call=None


In [4]:
tracer.spans

{eb/0e: SpanRecord(context=eb/0e, parent=None, error=None, record=None),
 eb/03: SpanMethodCall(context=eb/03, parent=eb/0e, error=None, call=None),
 eb/eb: SpanMethodCall(context=eb/eb, parent=eb/0e, error='test outer tracer error', call=None)}

In [5]:
inner_tracer.spans

{a5/66: SpanRecord(context=a5/66, parent=eb/0e, error=None, record=None),
 a5/e4: SpanMethodCall(context=a5/e4, parent=a5/66, error='test inner tracer error', call=None)}

In [6]:
from trulens_eval.trace import Tracer, TracerProvider, tracer_provider, get_tracer

tracer = get_tracer()
inner_tracer = get_tracer()

with tracer.method() as span1:
    print(type(span1), span1)

    with tracer.method() as span2:
        print(type(span2), span2)

    with inner_tracer.method() as span3:
        print(type(span3), span3)

null SpanMethodCall
<class 'trulens_eval.trace.SpanMethodCall'> context=6e/59 parent=None error=None call=None
null SpanMethodCall
<class 'trulens_eval.trace.SpanMethodCall'> context=6e/97 parent=6e/59 error=None call=None
null SpanMethodCall
<class 'trulens_eval.trace.SpanMethodCall'> context=6e/d7 parent=6e/59 error=None call=None


In [7]:
from concurrent.futures import as_completed
from time import sleep

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from tqdm.auto import tqdm

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.schema.feedback import FeedbackMode
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tp = TP()

d = Dummy(
    loading_prob=0.0,
    freeze_prob=0.0, # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=1000,
    alloc = 0, # how much fake data to allocate during requests
    delay = 10.0
)

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.16.1.117:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [8]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [9]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()
assert rec is not None

tracer SpanRecord
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
finished recording, have:
3b/87 SpanRecord
3b/8b SpanMethodCall
3b/0e SpanMethodCall
3b/dc SpanMethodCall
3b/5d SpanMethodCall
3b/4e SpanMethodCall
3b/56 SpanMethodCall
3b/0a SpanMethodCall


In [10]:
get_tracer().spans

[]

In [13]:
recorder.tracer[1].spans

{3b/87: SpanRecord(context=3b/87, parent=None, error=None, record=None),
 3b/8b: SpanMethodCall(context=3b/8b, parent=3b/87, error=None, call=None),
 3b/0e: SpanMethodCall(context=3b/0e, parent=3b/8b, error=None, call=None),
 3b/dc: SpanMethodCall(context=3b/dc, parent=3b/0e, error=None, call=None),
 3b/5d: SpanMethodCall(context=3b/5d, parent=3b/8b, error=None, call=None),
 3b/4e: SpanMethodCall(context=3b/4e, parent=3b/8b, error=None, call=None),
 3b/56: SpanMethodCall(context=3b/56, parent=3b/8b, error=None, call=None),
 3b/0a: SpanMethodCall(context=3b/0a, parent=3b/8b, error=None, call=None)}

In [15]:
recorder.tracer[1]